# Build a classifier nn

In [5]:
#Imports

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset

import torchvision
import torchvision.transforms as transforms

## Data

In [2]:
class NpzDataset(torch.utils.data.Dataset):
    # Class to create a torch Dataset class for inputting into Dataloader
    def __init__(self, data):
        self.img = data["images"]
        self.lab = data["labels"]

    def __getitem__(self, idx):
        return (self.img[idx], self.lab[idx])

    def __len__(self):
        return len(self.img)

In [3]:
data_train = np.load(r"/Users/alejandranavarrocastillo/Documents/GitHub/dtu_mlops_s1_final_exercise/data/corruptmnist/train.npz")
data_test = np.load(r"/Users/alejandranavarrocastillo/Documents/GitHub/dtu_mlops_s1_final_exercise/data/corruptmnist/test.npz")

# Create Datasets from the npz files
dataset_train = NpzDataset(data_train)
dataset_test = NpzDataset(data_test)

# Create dataloaders from the Datasets
trainloader = torch.utils.data.DataLoader(dataset_train, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(dataset_test, batch_size=64, shuffle=True)

## Define a convolutional nn

In [7]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

## Define a Loss function and optimizer

In [8]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Train the network

In [9]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

RuntimeError: Input type (double) and bias type (float) should be the same